In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import BBG_data_process_funcs as data_process
import risk_free_curve as rfcurve
from openpyxl import load_workbook

#### Pricing as of 21 Sep 2018

In [2]:
pdate = dt.datetime.strptime('2018/09/21', "%Y/%m/%d")
pdate_string = pdate.strftime('%d%b%Y')

#### Convertible Bond input data from Bloomberg

In [3]:
inputfile = 'AndersonBuffum_PricingInput_' + pdate_string + 'Final.xlsx'

cb_data = pd.read_excel(inputfile,sheet_name=0)
cb_data = cb_data.iloc[1:]
cb_data['MATURITY'] = pd.to_datetime(cb_data['MATURITY'])

cds_data = pd.read_excel(inputfile,sheet_name='CDS Curves')

call_put_schedule_data = pd.read_excel(inputfile,sheet_name='Call Put Schedule')
cp_schedules = data_process.process_call_put_schedule_data(call_put_schedule_data)

#### Risk free discount curve constructed from Bloomberg Data

In [10]:
rf_tenor_tickers = pd.read_excel(inputfile, sheet_name='IR Term TICKERS').set_index('TICKER')
rf_tenor_dict = rf_tenor_tickers['YEARS'].to_dict()

rf_rates_data = pd.read_excel(inputfile, sheet_name='IR Term Structure')
rf_rates_data = rf_rates_data.iloc[6:]
rf_rates_data.columns = np.insert(rf_rates_data.columns[1:],0,'Dates')
rf_rates_data = rf_rates_data.set_index('Dates')

def construct_rf_curve(pricing_date: dt.datetime):
    pdate_rf_rates = rf_rates_data.loc[pdate]
    rf_tenors = np.array([rf_tenor_dict[i] for i in pdate_rf_rates.index])
    rf_zrates = pdate_rf_rates.values/100.
    rf_curve = rfcurve.risk_free_curve(rf_tenors, rf_zrates)
    return rf_curve

rf_curve = construct_rf_curve(pdate)

####  Compare with Bloomberg Convertible Bond Data

In [18]:
def ABpricer_on_BBG_data(row, pdate, cds_data, cp_schedules, rf_curve, p, use_cds_term_structure=False):
    
    contract_info = {
        'maturity_date': row['MATURITY'],
        'coupon_rate': row['CPN'],
        'coupon_freq': row['CPN_FREQ'],
        'notional': 1000,
        'conv_ratio': None,  # will be derived from conv_price in the code
        'conv_price': row['CV_CNVS_PX'],
        'callable': row['CALLABLE'] == 'Y',
        'puttable': row['PUTABLE'] == 'Y',
    }

    if (contract_info['callable'] or contract_info['puttable']):
        ISIN = row['ISIN']
        if (ISIN in cp_schedules):
            contract_info['call_schedule'], contract_info['put_schedule'] = cp_schedules[ISIN]
        else:
            contract_info['call_schedule'], contract_info['put_schedule'] = (None, None)

    if 'SOFT CALL 20-30' in row:
        contract_info['softcall'] = row['SOFT CALL 20-30'] == 'Y'
    else:
        contract_info['softcall'] = False

    if (contract_info['softcall']):
        contract_info['softcall_start'] = pd.to_datetime(row['SOFT CALL START'])
        contract_info['softcall_end'] = pd.to_datetime(row['SOFT CALL END'])
        contract_info['softcall_barrier'] = row['SOFT CALL BARRIER']
        contract_info['softcall_redempt'] = row['SOFT CALL REDEMPTION']

    rr = row['BOND_RECOVERY_RATE']
    rr = 0.4 if np.isnan(rr) else rr

    if (use_cds_term_structure == True):
        credit_spread = parse_BBG_cds_spread(row['ISIN'], cds_data)
        credit_tenors = [0.5, 1, 2, 3, 4, 5, 7, 10]
    else:
        credit_spread = [row['FLAT_CREDIT_SPREAD_CV_MODEL']]
        credit_tenors = [5]

    model_params = {
        'recovery_rate': rr,
        'equity_spot': row['CV_MODEL_UNDL_PX'],
        'equity_dividend_yield': row['EQY_DVD_YLD_IND'] / 100.,
        'equity_flat_vol': row['CV_MODEL_STOCK_VOL'],
        'eta': row['STOCK_JUMP_ON_DEFAULT_CV_MODEL'],
        'credit tenors': credit_tenors,
        'credit spread': credit_spread, 
    }

    CB = ConvertibleBond(contract_info, model_params, rf_curve, p=p)
    ABpricer = AndersenBuffumPricer(CB, pdate, dt=1. / 48., dy=0.05)

    output = (ABpricer.clean_price(), ABpricer.eq_spot_delta(), ABpricer.eq_spot_gamma(), ABpricer.eq_vega())
    return output

In [111]:
# ------------------------------------------------------
# Crank-Nicolson Finite Difference Solver for
# Andersen-Buffum Convertible Bond model
# ------------------------------------------------------

import numpy as np
import datetime as dt
from ConvertibleBondClass import ConvertibleBond

# -----------------------------------------------------
# an efficient tridiagonal-matrix solver
# -----------------------------------------------------

def TDMA(a, b, c, d):
    # tridiagonal-matrix solver: a = Lower Diag, b = Main Diag, c = Upper Diag, d = solution vector
    n = len(d)
    w = np.zeros(n - 1, float)
    g = np.zeros(n, float)
    p = np.zeros(n, float)

    w[0] = c[0] / b[0]
    g[0] = d[0] / b[0]

    for i in range(1, n - 1):
        w[i] = c[i] / (b[i] - a[i - 1] * w[i - 1])
    for i in range(1, n):
        g[i] = (d[i] - a[i - 1] * g[i - 1]) / (b[i] - a[i - 1] * w[i - 1])
    p[n - 1] = g[n - 1]
    for i in range(n - 1, 0, -1):
        p[i - 1] = g[i - 1] - w[i - 1] * p[i]

    return p


# ------------------------------------------------------
# Crank-Nicolson Finite Difference Solver for
# Andersen-Buffum Convertible Bond model
# ------------------------------------------------------

class AndersenBuffumPricer():
    
    theta = 0.5  # keep it as 0.5 to use Crank-Nicolson Method
    y_grid_coverage = 4  # no of stddev covered by the y-grid

    def __init__(self, sec: ConvertibleBond, pdate: dt.datetime, dt=1 / 48, dy=0.05):
        
        self.dt = dt
        self.dy = dy
        self.sec = sec
        self.pdate = pdate

        # discretize time to maturity
        self.ttm = (sec.m_date - pdate).days / 365.25
        self.M = int(self.ttm / dt)  # round down to nearest integer
        self.T = self.M * dt
        self.rf_T = self.ttm - self.T
        print(sec.m_date, pdate, self.ttm, dt, self.M, self.T, self.rf_T)

        # discretize spatial mesh in log-price space
        self.y0 = np.log(sec.eq_spot)
        y_half_range = self.y_grid_coverage * sec.sigma(self.T) * np.sqrt(self.T)
        N_half = int(y_half_range/dy)
        self.N = int(2 * N_half)
        print(sec.eq_spot, self.y0, y_half_range, pdate, N_half, self.N)

        self.t_knots = np.arange(0, self.M + 1, 1) * self.dt
        self.y_knots = self.y0 + np.arange(-N_half, N_half + 0.5, 1) * self.dy
        self.S_knots = np.exp(self.y_knots)
        self.v_knots = np.array([])
        self.S0_index = N_half

        self._init_lambd()
        self._init_sigma_sq()
        self._init_K()
        self._init_L()
        self._init_R()  # recovery value of the bond upon default

        self._init_u()
        self._init_d()
        self._init_l()

        if (self.sec.callable):
            self.callable_grid = self._setup_callable_grid()

        if (self.sec.puttable):
            self.puttable_grid = self._setup_puttable_grid()

        if (self.sec.softcall):
            self.softcall_barrier = self.sec.soft_call_approx_1of1_barrier(pdate)
            self.softcall_barrier_index = np.argmax(self.S_knots >= self.softcall_barrier)
            self.softcall_end_tau = (self.sec.m_date - self.sec.softcallinfo['end']).days / 365.25
            self.softcall_start_tau = (self.sec.m_date - self.sec.softcallinfo['start']).days / 365.25

    def _setup_strike_grid(self, schedule):
        if schedule is None:
            return np.ones(self.M + 1) * self.sec.notional

        dates = schedule['date'].values
        prices = schedule['price'].values * (self.sec.notional / 100.)  # data was per 100 notional
        call_taus = [(self.sec.m_date - d).days / 365.25 for d in dates]
        call_grid = np.zeros(self.M + 1)
        for m in range(0, self.M + 1):
            for call_t, call_price in zip(call_taus, prices):
                if (self.t_knots[m] < call_t and call_t <= self.t_knots[m + 1]):
                    call_grid[m + 1] = call_price

        return call_grid

    def _setup_callable_grid(self):
        return self._setup_strike_grid(self.sec.call_schedule)

    def _setup_puttable_grid(self):
        return self._setup_strike_grid(self.sec.put_schedule)

    def reset_p(self, newp):
        self.sec.reset_p(newp)
        self.__init__(self.sec, self.pdate, self.dt, self.dy)

    def refresh_security(self, sec: ConvertibleBond):
        self.sec = sec
        self.__init__(self.sec, self.pdate, self.dt, self.dy)

    def coupon_stream(self):
        cp_dt = 1.0 / float(self.sec.c_freq)
        cp = self.sec.c_rate * cp_dt * self.sec.notional
        cp_stream = np.zeros(self.M + 1)
        # enumerate in backward time (to be compatible with solver)
        step = int(round(cp_dt / self.dt, 0))
        for m in np.arange(0, self.M + 1, step):
            cp_stream[m] = cp
        return cp_stream

    def accrued_interests(self):
        coupons = self.coupon_stream()
        accrued = np.zeros(self.M + 1)
        c_index = np.argwhere(coupons > 0).flatten()
        c_start = c_index
        c_end = np.append(c_index[1:] - 1, len(coupons) - 2)
        for start, end in zip(c_start, c_end):
            c = coupons[start]
            slots = end - start + 1.
            accrued[start:end + 1] = (c / slots) * np.arange(slots, 0, -1)
        accrued = accrued - coupons
        return accrued

    def initial_condition(self):
        eq = self.sec.cv_ratio * self.S_knots
        bond = self.sec.notional
        return np.maximum(eq, bond) + self.coupon_stream()[0]

    def bc_lower(self):
        recovery = np.array([self.sec.rr * self.sec.notional] * (self.M + 1))
        return recovery + self.accrued_interests()

    def bc_upper(self):
        eq = np.array([self.sec.cv_ratio * self.S_knots[-1]] * (self.M + 1))
        return eq + self.accrued_interests()
    
    def _init_lambd(self):
        lambd = np.zeros((self.M + 1, self.N + 1))
        for m, tau in np.ndenumerate(self.t_knots):
            t = self.ttm - tau  # tau is backward time here
            lambd[m] = self.sec.lambd(t, self.S_knots)
        self.lambd = lambd

    def _init_sigma_sq(self):
        self.sigma_sq = np.array([self.sec.sigma(self.ttm - tau) ** 2 for tau in self.t_knots])

    def _init_K(self):
        rt = np.array([self.sec.r(self.ttm - tau) for tau in self.t_knots])
        qt = np.array([self.sec.q(self.ttm - tau) for tau in self.t_knots])
        self.K = self.sec.eta * self.lambd + rt[:, np.newaxis] - qt[:, np.newaxis] \
        - 0.5 * self.sigma_sq[:, np.newaxis] 

    def _init_L(self):
        rt = np.array([self.sec.r(self.ttm - tau) for tau in self.t_knots])
        self.L = -self.lambd - rt[:, np.newaxis]

    def _init_R(self):
        R = np.ones((self.M + 1, self.N + 1))
        Bond_RR = self.sec.rr * self.sec.notional * R
        EQ_conv = (1 - self.sec.eta) * self.sec.cv_ratio * self.S_knots
        EQ_RR = np.repeat(np.array([EQ_conv]), self.M + 1, axis=0)
        self.R = np.maximum(Bond_RR, EQ_RR)

    def _init_u(self):
        u = self.K + self.sigma_sq[:, np.newaxis] / self.dy
        self.u = (0.5 * self.dt / self.dy) * u

    def _init_d(self):
        d = -self.L + (self.sigma_sq[:, np.newaxis] / (self.dy ** 2))
        self.d = self.dt * d

    def _init_l(self):
        l = self.K - self.sigma_sq[:, np.newaxis] / self.dy
        self.l = (0.5 * self.dt / self.dy) * l

    def solve(self):
        M = self.M
        N = self.N
        theta = self.theta

        coupons = self.coupon_stream()
        accrued = self.accrued_interests()
        eq_conv = self.sec.cv_ratio * self.S_knots

        bc_lower = self.bc_lower()
        bc_upper = self.bc_upper()
        v = self.initial_condition()  # (N+1)-array
        v_dt = self.initial_condition()  # (N+1)-array

        for m in range(0, M):
            z  = (1 - theta) * (self.u[m, 1:-1] * v[2:])
            z += (1 - (1 - theta) * self.d[m, 1:-1]) * v[1:-1]
            z -= (1 - theta) * self.l[m, 1:-1] * v[0:-2]
            z += self.dt * theta * (self.lambd[m + 1, 1:-1] * self.R[m + 1, 1:-1])
            z += self.dt * (1 - theta) * (self.lambd[m, 1:-1] * self.R[m, 1:-1])

            v[0] = bc_lower[m + 1]
            # v[N] = bc_upper[m+1]
            v[N] = 2 * v[N - 1] - v[N - 2]

            b = np.zeros(N - 1)
            b[0] = -theta * self.l[m + 1, 1] * v[0]
            b[-1] = theta * self.u[m + 1, N - 1] * v[N]

            C_l_diag = theta * self.l[m + 1, 2:-1]
            C_m_diag = 1 + theta * self.d[m + 1, 1:-1]
            C_u_diag = -theta * self.u[m + 1, 1:-2]

            # solve tridiagonal-matrix problem
            v[1:-1] = TDMA(C_l_diag, C_m_diag, C_u_diag, z + b)

            # 20-of-30 soft-call by issuer
            if (self.sec.softcall):
                tau = (m + 1) * self.dt
                if (tau >= self.softcall_end_tau and tau <= self.softcall_start_tau):
                    barx = self.softcall_barrier_index
                    v[barx:-1] = np.minimum(v[barx:-1], self.sec.softcallinfo['redemption'] + accrued[m + 1])

            # callable option by issuer has the least priority (iterating backward here)
            if (self.sec.callable and self.callable_grid[m + 1] > 0):
                call_price = self.callable_grid[m + 1]
                v[1:-1] = np.minimum(v[1:-1], call_price + accrued[m + 1])

            # puttable option by borrower (priority over issuer's call)
            if (self.sec.puttable and self.puttable_grid[m + 1] > 0):
                put_price = self.puttable_grid[m + 1]
                v[1:-1] = np.maximum(v[1:-1], put_price + accrued[m + 1])

            # borrower has option to convert any time (priority over issuer's call)
            v[1:-1] = np.maximum(v[1:-1], eq_conv[1:-1] + accrued[m + 1])

            # add discrete coupons - coupons are always paid before any conversion/call/put
            v[1:-1] = v[1:-1] + coupons[m + 1]

        # discount by the risk-free rate for the residual period smaller than dt
        self.v_knots = v * self.sec.Z(self.rf_T)

    def price(self):
        if (len(self.v_knots) == 0):
            self.solve()
        return self.v_knots[self.S0_index]

    def dirty_price(self):
        p = self.price()
        return p * (100. / self.sec.notional)

    def clean_price(self):
        dt = 1. / float(self.sec.c_freq)
        cp = self.sec.c_rate * dt * 100  #coupon per 100 notional
        remain = int(self.ttm / dt)
        accrued_t = dt - (self.ttm - remain * dt)
        accrued_c = cp * (accrued_t / dt)
        return self.dirty_price() - accrued_c

    def eq_spot_delta(self):
        self.solve()
        dvdy = (self.v_knots[self.S0_index + 1] - self.v_knots[self.S0_index - 1]) / (2 * self.dy)
        dvds = dvdy / self.sec.eq_spot
        delta = dvds / self.sec.cv_ratio  # same convention as  Bloomberg and BlackRock
        return delta

    def eq_spot_gamma(self):
        base_spot = self.sec.eq_spot
        abs_shock = min(10.0 / self.sec.cv_ratio, 0.01 * base_spot)

        self.sec.reset_eq_spot(base_spot + abs_shock)
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        up_delta = self.eq_spot_delta()

        self.sec.reset_eq_spot(base_spot - abs_shock)
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        dn_delta = self.eq_spot_delta()

        # revert to the base state
        self.sec.reset_eq_spot(base_spot)
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        return (up_delta - dn_delta) / (2 * abs_shock / base_spot)

    def eq_vega(self, shock=0.01):
        base_vol = self.sec.eq_vol

        self.sec.eq_vol = base_vol + shock
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        shocked_price = self.dirty_price()

        self.sec.eq_vol = base_vol
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        base_price = self.dirty_price()

        vega = (shocked_price - base_price)
        return vega

In [112]:
outputfile = 'AndersonBuffum_PricingOutput_' + pdate_string + 'Final.xlsx'
p = 0.0

In [115]:
#### Set 1) Pricing with Flat CDS spread

AB_model_data =[]

for index, value in cb_data.iterrows():
    results = ABpricer_on_BBG_data(value, pdate, cds_data,
                                                cp_schedules, rf_curve, p,
                                                use_cds_term_structure=False)
    print(value['ISIN'], results)
    AB_model_data.append(results)
    break

cols = ['ABM Price','ABM Delta','ABM Gamma','ABM Vega']
AB_model_df = pd.DataFrame(AB_model_data, columns=cols)

# data_to_save = data_process.reformat_output_data(AB_model_df, cb_data)
# writer = pd.ExcelWriter(outputfile)
# data_to_save.to_excel(writer,'CDS Flat Spread')
# writer.save()

2024-01-15 00:00:00 2018-09-21 00:00:00 5.316906228610541 0.020833333333333332 255 5.3125 0.00440622861054063
6.25 1.8325814637483102 4.410899102223041 2018-09-21 00:00:00 88 176
2024-01-15 00:00:00 2018-09-21 00:00:00 5.316906228610541 0.020833333333333332 255 5.3125 0.00440622861054063
6.3125 1.8425317946014783 4.410899102223041 2018-09-21 00:00:00 88 176
2024-01-15 00:00:00 2018-09-21 00:00:00 5.316906228610541 0.020833333333333332 255 5.3125 0.00440622861054063
6.1875 1.8225311278948086 4.410899102223041 2018-09-21 00:00:00 88 176
2024-01-15 00:00:00 2018-09-21 00:00:00 5.316906228610541 0.020833333333333332 255 5.3125 0.00440622861054063
6.25 1.8325814637483102 4.410899102223041 2018-09-21 00:00:00 88 176
2024-01-15 00:00:00 2018-09-21 00:00:00 5.316906228610541 0.020833333333333332 255 5.3125 0.00440622861054063
6.25 1.8325814637483102 4.50309454679597 2018-09-21 00:00:00 90 180
2024-01-15 00:00:00 2018-09-21 00:00:00 5.316906228610541 0.020833333333333332 255 5.3125 0.0044062286

In [16]:
#### Set 2) Pricing with CDS Spread term structure

AB_model_data2 =[]

for index, value in cb_data.iterrows():
    results = data_process.ABpricer_on_BBG_data(value,pdate,cds_data,cp_schedules,
                                                rf_curve, p,
                                                use_cds_term_structure=True)
    print(value['ISIN'], results)
    AB_model_data2.append(results)

cols = ['ABM Price','ABM Delta','ABM Gamma','ABM Vega']
AB_model_df2 = pd.DataFrame(AB_model_data2, columns=cols)

data_to_save2 = data_process.reformat_output_data(AB_model_df2, cb_data)

book = load_workbook(outputfile)
writer = pd.ExcelWriter(outputfile, engine = 'openpyxl')
writer.book = book
data_to_save2.to_excel(writer,'CDS Curve')
writer.save()
writer.close()

US62957HAB15 (76.23299872913695, 0.27011839744503957, 0.2825884018528235, 0.1718893850981118)
US25470MAD11 (85.33015028402079, 0.5687801558280812, 0.43612624369921416, 0.38139702659482566)
US04010LAT08 (102.29104804332727, 0.15434800450541747, 1.0954371945661165, 0.43546459224809553)
US267475AB73 (112.21430144649246, 0.5840797036523305, 0.6369498363428538, 0.5807553815581059)
US595112AY95 (153.8844898982779, 1.0000289983175346, 0.002419384748726618, 0.00032556374256387244)
US25155MKM28 (96.2521001287051, 0.45814273714465, 0.6980022934484742, 0.5282941962949508)
US698354AD99 (138.61944885014623, 0.7531828448923914, 0.26627334666270985, 0.6801363627658077)
US87157BAA17 (98.28606019430164, 0.03456466070573628, 0.06176452107617379, 0.0028634246696839227)
US966387AL67 (97.71215976056577, 0.03672236066747765, 0.13627311948268514, 0.030392322166505892)
US458140AF79 (245.51473431500435, 0.8346585147480529, 0.21674955381074856, 1.1598538744771645)
US88160RAB78 (105.39413733239226, 0.37886888354